In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.python.keras.layers import Dense
from numpy.random import default_rng
from sklearn.model_selection import train_test_split

In [2]:
# old_full_data = pd.read_csv('data/full_data.txt', sep=",", header=None, names=["e_t_x","e_t_y","e_t_z","e_r_x","e_r_y","e_r_z","e_r_w","d_t_x","d_t_y","d_t_z","d_r_x","d_r_y","d_r_z","d_r_w", "c_t_x","c_t_y","c_t_z","c_r_x","c_r_y","c_r_z", "c_r_w",  "speed"])
old_full_data = pd.read_csv('data/full_data_10HZ.csv', sep=",", header=None, names=["e_t_x","e_t_y","e_t_z","e_r_x","e_r_y","e_r_z","e_r_w","d_t_x","d_t_y","d_t_z","d_r_x","d_r_y","d_r_z","d_r_w", "c_t_x","c_t_y","c_t_z","c_r_x","c_r_y","c_r_z", "c_r_w",  "speed"])
# old_full_data
old_full_data.head(13)

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,stopped
1,-0.001,0.0,0.0,0.000,0.001,0.000,0.0,-0.001,0.000,0.000,...,0.0,0.000,0.000,0.0,0.000,0.001,0.000,0.000,0.000,stopped
2,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,stopped
3,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.001,0.000,...,0.0,-0.001,0.000,0.0,0.001,-0.001,0.000,0.001,0.000,stopped
4,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,stopped
5,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,-0.001,...,0.0,-0.001,0.000,0.0,0.001,0.000,0.000,0.001,0.000,stopped
6,0.000,0.0,0.0,0.001,0.000,0.000,0.0,0.000,0.000,0.001,...,0.0,0.000,0.000,0.0,0.000,0.001,-0.001,0.001,0.000,stopped
7,-0.001,0.0,0.0,-0.001,0.000,-0.001,0.0,0.000,0.000,0.000,...,0.0,0.000,0.000,0.0,0.000,0.001,0.000,0.000,-0.001,stopped
8,0.000,0.0,0.0,0.000,0.000,-0.001,0.0,0.000,0.000,0.000,...,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,stopped
9,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,...,0.0,0.000,0.001,0.0,0.000,-0.001,0.001,-0.001,0.000,stopped


In [81]:
# full_data = old_full_data.append(result, ignore_index=True)

/var/folders/0t/r6pd3gzs67n8rkyyt6l75dy80000gn/T/ipykernel_1413/3411656716.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = old_full_data.append(result, ignore_index=True)


In [82]:
# old_full_data[::6].to_csv('data/full_data_6.csv', index=False, header=True)

In [83]:
# full_data.to_csv('data/full_data2.txt', index=False)

In [84]:
# full_data.to_csv('data/full_data.txt', index=False, header=False)

In [85]:
# full_data.to_csv('data/full_data_100Hz.csv', index=False, header=False)

In [60]:
# result_df= full_data.loc[full_data['speed']=="fast"][1:].reset_index(drop=True)

In [ ]:
# result_df

In [49]:
data_stop_0= old_full_data.loc[old_full_data['speed']=="stopped"][1:].reset_index(drop=True)
data_stop_0

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.001,0.0,0.0,0.0,0.001,0.0,0.0,-0.001,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.0,stopped
1,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
2,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.001,0.000,...,0.000,-0.001,0.000,0.000,0.001,-0.001,0.000,0.001,0.0,stopped
3,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
4,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,-0.001,...,0.000,-0.001,0.000,0.000,0.001,0.000,0.000,0.001,0.0,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17992,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
17993,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
17994,0.000,0.0,0.0,0.0,-0.001,0.0,0.0,0.001,-0.003,0.001,...,0.003,-0.003,0.000,0.000,0.000,0.000,-0.002,0.001,0.0,stopped
17995,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped


In [48]:
data_stop_90_left_foot_without_negativation = swap_columns(data_stop_0, 'e_t_x', 'e_t_z')
data_stop_90_left_foot_without_negativation["e_t_z"] = data_stop_90_left_foot_without_negativation["e_t_z"]*-1
data_stop_90_left_foot=data_stop_90_left_foot_without_negativation
data_stop_90_left_right_foot_without_negativation = swap_columns(data_stop_90_left_foot, 'd_t_x', 'd_t_z')
data_stop_90_left_right_foot_without_negativation["d_t_z"] = data_stop_90_left_right_foot_without_negativation["d_t_z"]*-1
data_stop_90=data_stop_90_left_right_foot_without_negativation

data_stop_90.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_stop_90.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_stop_90.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_stop_90

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_z,d_t_y,d_t_x,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.0,0.0,-0.001,0.0,0.001,0.0,0.0,-0.000,0.000,-0.001,...,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.0,stopped
1,-0.0,0.0,0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
2,-0.0,0.0,0.000,0.0,0.000,0.0,0.0,-0.000,0.001,0.000,...,0.000,-0.001,0.000,0.000,0.001,-0.001,0.000,0.001,0.0,stopped
3,-0.0,0.0,0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
4,-0.0,0.0,0.000,0.0,0.000,0.0,0.0,0.001,0.000,0.000,...,0.000,-0.001,0.000,0.000,0.001,0.000,0.000,0.001,0.0,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17992,-0.0,0.0,0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
17993,-0.0,0.0,0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
17994,-0.0,0.0,0.000,0.0,-0.001,0.0,0.0,-0.001,-0.003,0.001,...,0.003,-0.003,0.000,0.000,0.000,0.000,-0.002,0.001,0.0,stopped
17995,-0.0,0.0,0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped


In [50]:
data_stop_180_left_foot_without_negativation = swap_columns(data_stop_90, 'e_t_x', 'e_t_z')
data_stop_180_left_foot_without_negativation["e_t_z"] = data_stop_180_left_foot_without_negativation["e_t_z"]*-1
data_stop_180_left_foot=data_stop_180_left_foot_without_negativation
data_stop_180_left_right_foot_without_negativation = swap_columns(data_stop_180_left_foot, 'd_t_x', 'd_t_z')
data_stop_180_left_right_foot_without_negativation["d_t_z"] = data_stop_180_left_right_foot_without_negativation["d_t_z"]*-1
data_stop_180=data_stop_180_left_right_foot_without_negativation

data_stop_180.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_stop_180.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_stop_180.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_stop_180

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.001,0.0,-0.0,0.0,0.001,0.0,0.0,-0.001,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.0,stopped
1,-0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
2,-0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.001,0.000,...,0.000,-0.001,0.000,0.000,0.001,-0.001,0.000,0.001,0.0,stopped
3,-0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
4,-0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.000,-0.001,...,0.000,-0.001,0.000,0.000,0.001,0.000,0.000,0.001,0.0,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17992,-0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
17993,-0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
17994,-0.000,0.0,-0.0,0.0,-0.001,0.0,0.0,0.001,-0.003,0.001,...,0.003,-0.003,0.000,0.000,0.000,0.000,-0.002,0.001,0.0,stopped
17995,-0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped


In [51]:
data_stop_270_left_foot_without_negativation = swap_columns(data_stop_180, 'e_t_x', 'e_t_z')
data_stop_270_left_foot_without_negativation["e_t_z"] = data_stop_270_left_foot_without_negativation["e_t_z"]*-1
data_stop_270_left_foot=data_stop_270_left_foot_without_negativation
data_stop_270_left_right_foot_without_negativation = swap_columns(data_stop_270_left_foot, 'd_t_x', 'd_t_z')
data_stop_270_left_right_foot_without_negativation["d_t_z"] = data_stop_270_left_right_foot_without_negativation["d_t_z"]*-1
data_stop_270=data_stop_270_left_right_foot_without_negativation

data_stop_270.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_stop_270.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_stop_270.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_stop_270

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_z,d_t_y,d_t_x,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.0,0.0,0.001,0.0,0.001,0.0,0.0,-0.000,0.000,-0.001,...,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.0,stopped
1,0.0,0.0,-0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
2,0.0,0.0,-0.000,0.0,0.000,0.0,0.0,-0.000,0.001,0.000,...,0.000,-0.001,0.000,0.000,0.001,-0.001,0.000,0.001,0.0,stopped
3,0.0,0.0,-0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
4,0.0,0.0,-0.000,0.0,0.000,0.0,0.0,0.001,0.000,0.000,...,0.000,-0.001,0.000,0.000,0.001,0.000,0.000,0.001,0.0,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17992,0.0,0.0,-0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
17993,0.0,0.0,-0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
17994,0.0,0.0,-0.000,0.0,-0.001,0.0,0.0,-0.001,-0.003,0.001,...,0.003,-0.003,0.000,0.000,0.000,0.000,-0.002,0.001,0.0,stopped
17995,0.0,0.0,-0.000,0.0,0.000,0.0,0.0,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped


In [52]:
data_stop_0_90 = data_stop_0.append(data_stop_90, ignore_index=True)
data_stop_0_90_180 = data_stop_0_90.append(data_stop_180, ignore_index=True)
data_stop = data_stop_0_90_180.append(data_stop_270, ignore_index=True)
data_stop

/var/folders/0t/r6pd3gzs67n8rkyyt6l75dy80000gn/T/ipykernel_980/2689906575.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_stop_0_90 = data_stop_0.append(data_stop_90, ignore_index=True)
/var/folders/0t/r6pd3gzs67n8rkyyt6l75dy80000gn/T/ipykernel_980/2689906575.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_stop_0_90_180 = data_stop_0_90.append(data_stop_180, ignore_index=True)
/var/folders/0t/r6pd3gzs67n8rkyyt6l75dy80000gn/T/ipykernel_980/2689906575.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_stop = data_stop_0_90_180.append(data_stop_270, ignore_index=True)


,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.001,0.0,0.0,0.0,0.001,0.0,0.0,-0.001,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.0,stopped
1,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
2,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.001,0.000,...,0.000,-0.001,0.000,0.000,0.001,-0.001,0.000,0.001,0.0,stopped
3,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
4,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000,-0.001,...,0.000,-0.001,0.000,0.000,0.001,0.000,0.000,0.001,0.0,stopped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71983,0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.000,-0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
71984,0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.000,-0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped
71985,0.000,0.0,-0.0,0.0,-0.001,0.0,0.0,0.001,-0.003,-0.001,...,0.003,-0.003,0.000,0.000,0.000,0.000,-0.002,0.001,0.0,stopped
71986,0.000,0.0,-0.0,0.0,0.000,0.0,0.0,0.000,0.000,-0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,stopped


In [39]:
data_0_slow= old_full_data.loc[old_full_data['speed']=="slow"][1:].reset_index(drop=True)
data_0_fast=old_full_data.loc[old_full_data['speed']=="fast"][1:].reset_index(drop=True)

In [40]:
data_0 = data_0_slow.append(data_0_fast, ignore_index=True)
data_0_base= data_0

/var/folders/0t/r6pd3gzs67n8rkyyt6l75dy80000gn/T/ipykernel_980/1777011274.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_0 = data_0_slow.append(data_0_fast, ignore_index=True)


In [25]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df


In [33]:
data_90_left_foot_without_negativation = swap_columns(data_0, 'e_t_x', 'e_t_z')
data_90_left_foot_without_negativation["e_t_z"] = data_90_left_foot_without_negativation["e_t_z"]*-1
data_90_left_foot=data_90_left_foot_without_negativation
data_90_left_right_foot_without_negativation = swap_columns(data_90_left_foot, 'd_t_x', 'd_t_z')
data_90_left_right_foot_without_negativation["d_t_z"] = data_90_left_right_foot_without_negativation["d_t_z"]*-1
data_90=data_90_left_right_foot_without_negativation

data_90.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_90.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_90.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_90

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_z,d_t_y,d_t_x,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,slow
1,-0.000,0.000,0.000,-0.001,-0.001,0.002,-0.001,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,-0.001,0.000,-0.001,0.000,slow
2,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,slow
3,0.001,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,slow
4,-0.000,0.000,0.000,0.000,0.001,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,slow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,0.016,-0.003,-0.002,0.000,0.006,-0.005,0.005,-0.000,0.000,0.000,...,-0.001,-0.001,-0.003,0.001,-0.001,-0.002,0.004,-0.002,0.000,fast
23993,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,fast
23994,0.006,0.000,0.001,-0.002,-0.012,0.011,0.001,0.003,0.000,0.000,...,0.016,-0.020,-0.004,0.000,-0.008,0.001,0.000,-0.001,0.000,fast
23995,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,fast


In [34]:
data_180_left_foot_without_negativation = swap_columns(data_90, 'e_t_x', 'e_t_z')
data_180_left_foot_without_negativation["e_t_z"] = data_180_left_foot_without_negativation["e_t_z"]*-1
data_180_left_foot=data_180_left_foot_without_negativation
data_180_left_right_foot_without_negativation = swap_columns(data_180_left_foot, 'd_t_x', 'd_t_z')
data_180_left_right_foot_without_negativation["d_t_z"] = data_180_left_right_foot_without_negativation["d_t_z"]*-1
data_180=data_180_left_right_foot_without_negativation

data_180.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_180.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_180.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_180

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_x,d_t_y,d_t_z,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,slow
1,-0.000,0.000,-0.000,-0.001,-0.001,0.002,-0.001,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,-0.001,0.000,-0.001,0.000,slow
2,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,slow
3,-0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,slow
4,-0.000,0.000,-0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,slow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,0.002,-0.003,0.016,0.000,0.006,-0.005,0.005,0.000,0.000,0.000,...,-0.001,-0.001,-0.003,0.001,-0.001,-0.002,0.004,-0.002,0.000,fast
23993,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,fast
23994,-0.001,0.000,0.006,-0.002,-0.012,0.011,0.001,0.000,0.000,-0.003,...,0.016,-0.020,-0.004,0.000,-0.008,0.001,0.000,-0.001,0.000,fast
23995,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,fast


In [35]:
data_270_left_foot_without_negativation = swap_columns(data_180, 'e_t_x', 'e_t_z')
data_270_left_foot_without_negativation["e_t_z"] = data_270_left_foot_without_negativation["e_t_z"]*-1
data_270_left_foot=data_270_left_foot_without_negativation
data_270_left_right_foot_without_negativation = swap_columns(data_270_left_foot, 'd_t_x', 'd_t_z')
data_270_left_right_foot_without_negativation["d_t_z"] = data_270_left_right_foot_without_negativation["d_t_z"]*-1
data_270=data_270_left_right_foot_without_negativation

data_270.rename(columns={'e_t_z':'e_t_x2'}, inplace=True)
data_270.rename(columns={'e_t_x':'e_t_z'}, inplace=True)
data_270.rename(columns={'e_t_x2':'e_t_x'}, inplace=True)
data_270

,e_t_x,e_t_y,e_t_z,e_r_x,e_r_y,e_r_z,e_r_w,d_t_z,d_t_y,d_t_x,...,d_r_z,d_r_w,c_t_x,c_t_y,c_t_z,c_r_x,c_r_y,c_r_z,c_r_w,speed
0,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,slow
1,0.000,0.000,-0.000,-0.001,-0.001,0.002,-0.001,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,-0.001,0.000,-0.001,0.000,slow
2,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,slow
3,-0.001,0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,slow
4,0.000,0.000,-0.000,0.000,0.001,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,slow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23992,-0.016,-0.003,0.002,0.000,0.006,-0.005,0.005,-0.000,0.000,0.000,...,-0.001,-0.001,-0.003,0.001,-0.001,-0.002,0.004,-0.002,0.000,fast
23993,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,fast
23994,-0.006,0.000,-0.001,-0.002,-0.012,0.011,0.001,0.003,0.000,0.000,...,0.016,-0.020,-0.004,0.000,-0.008,0.001,0.000,-0.001,0.000,fast
23995,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,fast


In [53]:
data_stop.to_csv('stop_90.csv', index=False, header=True)
data_0.to_csv('0_90.csv', index=False, header=True)
data_90.to_csv('90_90.csv', index=False, header=True)
data_180.to_csv('180_90.csv', index=False, header=True)
data_270.to_csv('270_90.csv', index=False, header=True)